In [ ]:
from common import *

use_hvplot()

In [ ]:
client = start_cluster(4, 20)
client.dashboard_link.split('/')[2]

# Precipitation input comparison between iSnobal and CBRFC Snow-17 (HRRR F06)

## CBRFC

In [ ]:
CBRFC_precip = pd.read_csv(
    CBRFC_DIR / 'precip_Alec2.csv',
    parse_dates=True,
    index_col=0,
    header=0,
    names=['ALEC2HLF', 'ALEC2HMF', 'ALEC2HUF', 'Average'],
    dtype={'ALEC2HLF': np.float64, 'ALEC2HMF': np.float64, 'ALEC2HUF': np.float64},
)
CBRFC_precip.ALEC2HLF *= INCH_TO_MM
CBRFC_precip.ALEC2HMF *= INCH_TO_MM
CBRFC_precip.ALEC2HUF *= INCH_TO_MM

## iSnobal 

In [ ]:
wy_precip = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy20*' / 'erw/*/precip.nc').as_posix(),
    parallel=True,
).resample(**RESAMPLE_1_DAY_OPTS).sum()

wy_precip.coords['mask'] = (('y', 'x'), cbrfc_zones())

In [ ]:
hlf = wy_precip.where(wy_precip.mask == ALEC2HLF).precip.mean(("x", "y")).compute()
hmf = wy_precip.where(wy_precip.mask == ALEC2HMF).precip.mean(("x", "y")).compute()
huf = wy_precip.where(wy_precip.mask == ALEC2HUF).precip.mean(("x", "y")).compute()

### Treating all iSnobal values of less than 1 mm as no precipitation 

In [ ]:
hlf[hlf < 1] = 0
hmf[hmf < 1] = 0
huf[huf < 1] = 0

## Precipitation 

In [ ]:
huf.hvplot(label='ALEC HUF (iSnobal)', color='indigo', alpha=0.6). \
    opts(
        title='iSnobal versus Snow-17 CBRFC Precipitation', ylabel='Precipitation (mm)',
        yformatter='%d',
        width=1280, height=720
    ) * \
hmf.hvplot(label='ALEC HMF (iSnobal)', color='teal', alpha=0.6) * \
hlf.hvplot(label='ALEC HLF (iSnobal)', color='gold', alpha=0.6) * \
CBRFC_precip.ALEC2HUF.plot(color='indigo', line_dash='dashed', alpha=0.6) * \
CBRFC_precip.ALEC2HMF.plot(color='teal', line_dash='dashed', alpha=0.6) * \
CBRFC_precip.ALEC2HLF.plot(color='gold', line_dash='dashed', alpha=0.6)

In [ ]:
(CBRFC_precip.ALEC2HUF / huf).replace([np.nan, np.inf], 0).hvplot(label='ALEC HUF', color='indigo', alpha=0.6). \
    opts(
        title='Snow-17/iSnobal Precipitation Ratios',
        yformatter='%d',
        width=1280, height=720
    ) * \
(CBRFC_precip.ALEC2HMF / hmf).replace([np.nan, np.inf], 0).hvplot(label='ALEC HMF', color='teal', alpha=0.6) * \
(CBRFC_precip.ALEC2HLF / hlf).replace([np.nan, np.inf], 0).hvplot(label='ALEC HLF', color='gold', alpha=0.6)

### iSnobal 

In [ ]:
huf.hvplot(label='ALEC HUF (iSnobal)', color='indigo', alpha=0.6). \
    opts(
        title='iSnobal Precipitation', ylabel='Precipitation (mm)',
        yformatter='%d',
        width=1280, height=640
    ) * \
hmf.hvplot(label='ALEC HMF (iSnobal)', color='teal', alpha=0.6) * \
hlf.hvplot(label='ALEC HLF (iSnobal)', color='gold', alpha=0.6)

### CBRFC 

In [ ]:
CBRFC_precip.ALEC2HUF.plot(color='indigo', line_dash='dashed', alpha=0.6). \
    opts(title='Snow-17 CBRFC Precipitation', ylabel='Precipitation (mm)', width=1280, height=640) * \
CBRFC_precip.ALEC2HMF.plot(color='teal', line_dash='dashed', alpha=0.6) * \
CBRFC_precip.ALEC2HLF.plot(color='gold', line_dash='dashed', alpha=0.6)

## Yearly statistics 

### Total precipitation 

In [ ]:
stats = np.zeros([9,4])
start_year = 2017

for y_index in range(0, stats.shape[1]):
    year = start_year + y_index
    time=slice(f"{year}-10-01", f"{year + 1}-09-30")
    
    stats[0, y_index] = huf.sel(time=time).sum()
    stats[1, y_index] = CBRFC_precip.ALEC2HUF[time].sum()
    stats[2, y_index] = stats[0, y_index] / stats[1, y_index]

    stats[3, y_index] = hmf.sel(time=time).sum()
    stats[4, y_index] = CBRFC_precip.ALEC2HMF[time].sum()
    stats[5, y_index] = stats[3, y_index] / stats[4, y_index]
    
    stats[6, y_index] = hlf.sel(time=time).sum()
    stats[7, y_index] = CBRFC_precip.ALEC2HLF[time].sum()
    stats[8, y_index] = stats[6, y_index] / stats[7, y_index]

In [ ]:
results = pd.DataFrame(
    stats, 
    columns=range(2018, 2022), 
    index=pd.MultiIndex.from_product([
        ['Upper', 'Middle', 'Lower'], 
        ['iSnobal', 'Snow-17', 'iSnobal/Snow-17']
    ])
)

ratios = pd.IndexSlice[:, 'iSnobal/Snow-17', :]
results.style.format('{:.2f} mm').format('{:.2%}', subset=ratios)

In [ ]:
def plot_year(year):
    time=slice(f"{year-1}-10-01", f"{year}-09-30")
    box_opts = dict(ylim=(-2, 12), ylabel='Percent (%)', box_fill_alpha=0.4, violin_fill_alpha=0.4,width=600)

    box_stats = hv.Violin(
        (CBRFC_precip.ALEC2HUF / huf).replace([np.inf], np.nan)[time].values, 
        group=str(year), label="HUF"
    ).opts(violin_fill_color='indigo', **box_opts) * \
    hv.Violin(
        (CBRFC_precip.ALEC2HMF / hmf).replace([np.inf], np.nan)[time].values, 
        group=str(year), label="HMF"
    ).opts(violin_fill_color='teal', **box_opts) * \
    hv.Violin(
        (CBRFC_precip.ALEC2HLF / hlf).replace([np.inf], np.nan)[time].values, 
        group=str(year), label="HLF"
    ).opts(violin_fill_color='gold', **box_opts)

    return box_stats

(
    hv.Layout(plot_year(2018)) + \
    hv.Layout(plot_year(2019)) + \
    hv.Layout(plot_year(2020)) + \
    hv.Layout(plot_year(2021))
).cols(2).opts(title="Ratio Snow-17/iSnobal", shared_axes=False)

In [ ]:
def plot_year(isnobal, zone, color):
    box_stats = []
    for year in range(2018, 2022):
        time=slice(f"{year-1}-10-01", f"{year}-09-30")
        box_opts = dict(ylim=(-0.5, 8), ylabel='Percent (%)', box_fill_alpha=0.5, whisker_color='gray', width=600)

        box_stats.append(
            hv.BoxWhisker(
                (CBRFC_precip[f"ALEC2{zone}"] / isnobal).replace([np.inf], np.nan)[time].values, 
                group=zone, label=str(year)
            ).opts(box_fill_color=color, **box_opts)
        )

    return box_stats[0] * box_stats[1] * box_stats[2] * box_stats[3]

(
    hv.Layout(plot_year(huf, "HUF", 'indigo')) + \
    hv.Layout(plot_year(hmf, "HMF", 'teal')) + \
    hv.Layout(plot_year(hlf, "HLF", 'gold'))
).cols(1).opts(title="Ratio Snow-17/iSnobal", shared_axes=False)

In [ ]:
client.shutdown()